In [14]:
import heapq
import random
import math

In [ ]:
#Q1

In [3]:
board = [
    ['r', 'n', 'b', 'q', 'k', 'b', 'n', 'r'],
    ['p', 'p', 'p', 'p', 'p', 'p', 'p', 'p'],
    ['.', '.', '.', '.', '.', '.', '.', '.'],
    ['.', '.', '.', '.', '.', '.', '.', '.'],
    ['.', '.', '.', '.', '.', '.', '.', '.'],
    ['.', '.', '.', '.', '.', '.', '.', '.'],
    ['P', 'P', 'P', 'P', 'P', 'P', 'P', 'P'],
    ['R', 'N', 'B', 'Q', 'K', 'B', 'N', 'R']
]

In [11]:
# Small letters for black and capital letters for White
pieceVals = {
    'p': 1, 'r': 5, 'n': 3, 'b': 3, 'q': 9, 'k': 999, 
    'P': 1, 'R': 5, 'N': 3, 'B': 3, 'Q': 9, 'K': 999  
}

In [5]:
def lMoves(board, color):
    moves = []
    for row in range(8):
        for col in range(8):
            piece = board[row][col]
            if piece == '.':
                continue
            if (color == 'white' and piece.isupper()) or (color == 'black' and piece.islower()):
                if piece.lower() == 'p':
                    moves += pMoves(board, row, col, piece)
                elif piece.lower() == 'n':  
                    moves += kMoves(board, row, col, piece)
                elif piece.lower() == 'r':
                    moves += rMoves(board, row, col, piece)
    return moves

In [6]:
def pMoves(board, row, col, piece):
    moves = []
    direction = 1 if piece.isupper() else -1  
    if 0 <= row + direction < 8:
        if board[row + direction][col] == '.':
            moves.append(((row, col), (row + direction, col)))
        if 0 <= col + 1 < 8 and board[row + direction][col + 1] != '.' and board[row + direction][col + 1].islower() != piece.islower():
            moves.append(((row, col), (row + direction, col + 1)))  
        if 0 <= col - 1 < 8 and board[row + direction][col - 1] != '.' and board[row + direction][col - 1].islower() != piece.islower():
            moves.append(((row, col), (row + direction, col - 1)))
    return moves

In [7]:
def kMoves(board, row, col, piece):
    knightaval = [(-2, -1), (-1, -2), (1, -2), (2, -1), (2, 1), (1, 2), (-1, 2), (-2, 1)]
    moves = []
    for x, y in knightaval:
        nr, nc = row + x, col + y
        if 0 <= nr < 8 and 0 <= nc < 8:
            if board[nr][nc] == '.' or board[nr][nc].islower() != piece.islower():
                moves.append(((row, col), (nr, nc)))
    return moves

In [8]:
def rMoves(board, row, col, piece):
    moves = []
    directions = [(-1, 0), (1, 0), (0, -1), (0, 1)] 
    for x, y in directions:
        r, c = row, col
        while True:
            r, c = r + x, c + y
            if 0 <= r < 8 and 0 <= c < 8:
                if board[r][c] == '.':
                    moves.append(((row, col), (r, c)))
                elif board[r][c].islower() != piece.islower():
                    moves.append(((row, col), (r, c)))  # Capture
                    break
                else:
                    break
            else:
                break
    return moves

In [9]:
def eval(board):
    ev = 0
    for row in range(8):
        for col in range(8):
            piece = board[row][col]
            if piece != '.':
                ev += pieceVals.get(piece, 0)
    return ev

In [12]:
def bs(board, depth, color, bWidth=2):
    beam = [(eval(board), board, [])]  

    while beam:
        candidates = []
        for evalScore, curr, mSeq in beam:
            if len(mSeq) == depth:
                return mSeq, evalScore

            lmoves = lMoves(curr, color)
            for move in lmoves:
                nboard = [row[:] for row in curr]  
                (sr, sc), (er, ec) = move
                piece = nboard[sr][sc]
                nboard[sr][sc] = '.'  
                nboard[er][ec] = piece
                nmSeq = mSeq + [move]
                meval = eval(nboard)
                candidates.append((meval, nboard, nmSeq))

        beam = heapq.nsmallest(bWidth, candidates, key=lambda x: x[0])

    return [], float('-inf')  

dlimit = 3 
bWidth = 3

best, score = bs(board, dlimit, color='white', bWidth=bWidth)

if best:
    print(f"Best move sequence: {best} with evaluation score: {score}")
else:
    print("No best move found.")

Best move sequence: [((7, 1), (5, 0)), ((5, 0), (3, 1)), ((3, 1), (1, 0))] with evaluation score: 2075


In [15]:
#Q2

In [23]:
def calDistance(p1, p2):
    return math.sqrt((p1[0] - p2[0]) ** 2 + (p1[1] - p2[1]) ** 2)

In [34]:
def total(route):
    tDistance = 0
    for i in range(len(route) - 1):
        tDistance += calDistance(route[i], route[i + 1])
    tDistance += calDistance(route[-1], route[0])
    return tDistance

In [35]:
def getNeighbors(route):
    neighbors = []
    n = len(route)
    for i in range(n):
        for j in range(i + 1, n):
            new = route[:]
            new[i], new[j] = new[j], new[i]
            neighbors.append(new)
    return neighbors

In [36]:
def hillClimb(coordinates):
    currR = coordinates[:]
    random.shuffle(currR)
    currD = total(currR)
    
    while True:
        neighbors = getNeighbors(currR)
        nextR = None
        Ndistance = currD
        
        for neighbor in neighbors:
            nDistance = total(neighbor)
            if nDistance < Ndistance:
                nextR = neighbor
                Ndistance = nDistance
                break 
        
        if Ndistance >= currD:
            break
        
        # Move to the better neighbor
        currR = nextR
        currD = Ndistance
    
    return currR, currD

In [40]:
coordinates = [
    (0, 0), (1, 2), (4, 3), (6, 5), (7, 8), (9, 1), (10, 4), (3, 6)
]

op, tDistance = hillClimb(coordinates)

# Print the results
print("Optimized delivery route:")
for point in op:
    print(f"({point[0]}, {point[1]})")

print(f"Total distance covered: {tDistance}")

Optimized delivery route:
(7, 8)
(6, 5)
(3, 6)
(1, 2)
(0, 0)
(4, 3)
(9, 1)
(10, 4)
Total distance covered: 31.580201720139012


In [43]:
cities = [(0, 0), (1, 3), (3, 1), (5, 4), (6, 0), (7, 2), (8, 8), (2, 6), (4, 7), (6, 3)]
num = len(cities)

def calDistance(c1, c2):
    return math.sqrt((c1[0] - c2[0]) ** 2 + (c1[1] - c2[1]) ** 2)

def eval(route):
    tDistance = 0
    for i in range(len(route) - 1):
        tDistance += calDistance(cities[route[i]], cities[route[i + 1]])
    tDistance += calDistance(cities[route[-1]], cities[route[0]]) 
    return tDistance

def create():
    route = list(range(num))
    random.shuffle(route)
    return route

def selection(population, fitnessScores):
    sortedd = [x for _, x in sorted(zip(fitnessScores, population))]
    return sortedd[:len(population) // 2]

def crossover(parent1, parent2):
    start, end = sorted(random.sample(range(num), 2))
    child = [-1] * num
    child[start:end+1] = parent1[start:end+1]
    
    currpos = 0
    for city in parent2:
        if city not in child:
            while child[currpos] != -1:
                currpos += 1
            child[currpos] = city
    return child

def mutate(route):
    i, j = random.sample(range(num), 2) 
    route[i], route[j] = route[j], route[i] 
    return route

def genetic(popSize=100, mRate=0.1, maxGenz=1000):
    population = [create() for _ in range(popSize)] 
    
    for generation in range(maxGenz):
        fitnessScores = [eval(route) for route in population]
        
        bestFitness = min(fitnessScores)
        bestRoute = population[fitnessScores.index(bestFitness)]
        print(f"Generation {generation + 1}, Best Fitness: {bestFitness}")
        
        parents = selection(population, fitnessScores)
        
        new = []
        while len(new) < popSize:
            parent1, parent2 = random.sample(parents, 2)
            child = crossover(parent1, parent2)
            if random.random() < mRate:
                child = mutate(child)
            new.append(child)
        
        population = new 

    fitnessScores = [eval(route) for route in population]
    bestRoute = population[fitnessScores.index(min(fitnessScores))]
    return bestRoute, min(fitnessScores)

bestRoute, bestFitness = genetic(popSize=100, mRate=0.05, maxGenz=500)

print("\nBest route found:")
print("Route:", best_route)
print("Cities in order of visit:")
for city_idx in best_route:
    print(f"City {city_idx + 1}: {cities[city_idx]}")
print(f"Total distance: {best_fitness}")


Generation 1, Best Fitness: 39.247588140387215
Generation 2, Best Fitness: 38.55383553901659
Generation 3, Best Fitness: 36.876043330992786
Generation 4, Best Fitness: 34.18460192925768
Generation 5, Best Fitness: 34.8095604014353
Generation 6, Best Fitness: 34.18460192925768
Generation 7, Best Fitness: 35.94277800868117
Generation 8, Best Fitness: 36.2227594618988
Generation 9, Best Fitness: 35.013029054003866
Generation 10, Best Fitness: 33.70058484029074
Generation 11, Best Fitness: 35.013029054003866
Generation 12, Best Fitness: 32.18460192925768
Generation 13, Best Fitness: 31.90120647078314
Generation 14, Best Fitness: 29.072779346036945
Generation 15, Best Fitness: 29.072779346036945
Generation 16, Best Fitness: 29.072779346036945
Generation 17, Best Fitness: 29.072779346036945
Generation 18, Best Fitness: 29.072779346036945
Generation 19, Best Fitness: 29.072779346036945
Generation 20, Best Fitness: 29.072779346036945
Generation 21, Best Fitness: 29.072779346036945
Generation 2

NameError: name 'best_route' is not defined